In [8]:
from selenium.common import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

driver = webdriver.Chrome()
driver.set_window_size(1024, 600)
driver.maximize_window()

driver.get("https://www.google.fr/maps/place/Tour+Eiffel/@48.8583701,2.2919064,17z/data=!3m2!4b1!5s0x47e66fe1ee0293a1:0x213fe992eb6cca0c!4m6!3m5!1s0x47e66e2964e34e2d:0x8ddca9ee380ef7e0!8m2!3d48.8583701!4d2.2944813!16zL20vMDJqODE?entry=ttu")
names = []
rating = []
dates = []
comments = []

try:
    # Wait for consent page to load
    accept_all_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[jsname='b3VHJd']"))
    )
    # Accept cookies because they are good
    accept_all_button.click()
except:
    print("An error occurred when trying to accept Google's terms of conditions.")

# get to reviews
try:
    reviews_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[aria-label^='Reviews for']"))
    )
    reviews_button.click()
except:
    print("An error occurred when trying to get to reviews.")

# scroll until the end
try:
    scrollable_div = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf"))
    )
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    print(last_height)
    while True:
        # scroll to the bottom
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div
        )
        # wait for the page to load
        time.sleep(2)

        # calculate new scroll height and compare with the last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        print("newheight", new_height, "lastheight", last_height)
        if new_height == last_height:
            break
        last_height = new_height
except:
    print("Couldn't scroll...")

# open the comments fully (see more)
try:
    # gather all see more buttons and click them
    all_see_more_buttons = driver.find_elements(By.CSS_SELECTOR, "[aria-label = 'See more'")
    for button in all_see_more_buttons:
        button.click()
except:
    print("Couldn't click all see more buttons.")

# extract text
try:
    comment_divs = driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium ")
    # there is necessarily a name, rating and a date but not comment
    for comment in comment_divs:
        names.append(comment.find_element(By.CLASS_NAME, "d4r55 ").text)
        rating.append(comment.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label"))
        dates.append(comment.find_element(By.CLASS_NAME, "rsqaWe").text)
        # verify that there is comment and add, else add empty string
        text = ""
        try:
            myened_element = comment.find_element(By.CLASS_NAME, "MyEned")
            text = myened_element.find_element(By.CLASS_NAME, "wiI7pd").text
        except NoSuchElementException:
            text = ""
        comments.append(text)
except:
    print("Something happened when trying to parse reviews...")

finally:
    driver.quit()

zipped = list(zip(names, rating, dates, comments))
dictionary = {key: (v1, v2, v3) for key, v1, v2, v3 in zipped}

print(dictionary)

An error occurred when trying to get to reviews.
Couldn't scroll...
{'': ('5\xa0étoiles', '', '')}
